In [ ]:
import os
import openai
import tiktoken
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from rank_bm25 import BM25Okapi
from kiwipiepy import Kiwi

# Set OpenAI API key
os.environ["OPENAI_API_KEY"] = 'API_KEY'

# Tokenizer setup
tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

# Load and process PDF
file = "DATA_PATH"
loader = PyPDFLoader(file)
pages = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100, length_function=tiktoken_len)
texts = text_splitter.split_documents(pages)

# Tokenize texts using Kiwi
kiwi = Kiwi()
tokenized_texts = [[token[0] for token in kiwi.tokenize(text.page_content)] for text in texts]

# BM25 setup
bm25 = BM25Okapi(tokenized_texts)

def search(query, k=5):
    tokenized_query = [token[0] for token in kiwi.tokenize(query)]
    scores = bm25.get_scores(tokenized_query)
    sorted_scores = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
    top_k = sorted_scores[:k]
    return [(texts[i], score) for i, score in top_k]

# Embeddings setup
model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Vector store setup
docsearch = Chroma.from_documents(texts, hf)

# OpenAI model setup for retrieval QA
openai_llm = ChatOpenAI(model_name="gpt-4", streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=0)

qa = RetrievalQA.from_chain_type(llm=openai_llm,
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever(search_type="mmr", search_kwargs={'k': 3, 'fetch_k': 10}),
                                 return_source_documents=True)

# Prompt for generating questions and answers
class QAPair(BaseModel):
    question: str = Field(alias='QUESTION')
    answer: str = Field(alias='ANSWER')

prompt_template = PromptTemplate.from_template(
    """Context information is below. You are only aware of this context and nothing else.
    ---------------------
    {context}
    ---------------------
    Given this context, generate only questions based on the below query.
    You are an Teacher/Professor in {domain}. 
    Your task is to provide exactly **{num_questions}** question(s) for an upcoming quiz/examination. 
    You are not to provide more or less than this number of questions. 
    The question(s) should be diverse in nature across the document. 
    The purpose of question(s) is to test the understanding of the students on the context information provided.
    You must also provide the answer to each question. The answer should be based on the context information provided only.
    Restrict the question(s) to the context information provided only.
    QUESTION and ANSWER should be written in Korean. response in JSON format which contains the `question` and `answer`.
    ANSWER should be a complete sentence.
    #Format:
    ```json
    {{
    "CONTEXT": "정부는 방사능 오염에 대한 위험을 최소화하기 위해 일본 동부 6개현에서 출항하는 선박의 평형수 관리 규정을 강화했습니다. 특히, 일본 동부 6개현에서 평형수를 주입한 선박이 국내로 입항할 경우, 입항 24시간 전까지 평형수 입항보고서를 제출하도록 의무화하였습니다. 또한, 일본 동부 6개현에서 평형수를 주입한 후 이를 국내에서 배출할 예정인 선박은 반드시 우리나라 관할수역 밖에서 평형수를 교환해야 합니다. 이러한 조치는 방사능 오염 물질이 국내 해역에 유입되는 것을 방지하기 위해 마련된 것입니다.정부는 방사능 오염에 대한 위험을 최소화하기 위해 일본 동부 6개현에서 출항하는 선박의 평형수 관리 규정을 강화했습니다. 특히, 일본 동부 6개현에서 평형수를 주입한 선박이 국내로 입항할 경우, 입항 24시간 전까지 평형수 입항보고서를 제출하도록 의무화하였습니다. 또한, 일본 동부 6개현에서 평형수를 주입한 후 이를 국내에서 배출할 예정인 선박은 반드시 우리나라 관할수역 밖에서 평형수를 교환해야 합니다. 이러한 조치는 방사능 오염 물질이 국내 해역에 유입되는 것을 방지하기 위해 마련된 것입니다.",
    "QUESTION": "방사능 오염 선박평형수가 국내에 유입되지 않도록 어떤 조치가 시행되고 있습니까?",
    "ANSWER": "일본 동부 6개현에서 평형수를 주입 후 국내 입항 예정인 선박은 국내 입항 24시간 전까지 평형수 입항보고서를 제출해야 하며, 일본 동부 6개현에서 평형수를 주입 후 국내 배출 예정인 선박은 우리나라 관할수역 밖에서 평형수 교환 후 입항해야 합니다."
    }},
    {{
    "CONTEXT": "지방해양수산청은 선박의 출항 절차를 엄격히 관리하고 있습니다. 선박이 출항하기 전까지 제출된 모든 자료를 면밀히 검토하여, 선박이 평형수를 적절히 배출하지 않았는지 여부를 확인합니다. 이러한 검토 과정을 통해 평형수에 포함된 오염 물질이 국내 해역으로 유입되는 것을 방지합니다. 따라서, 선박의 출항 허가는 평형수 배출 여부가 확인된 후에만 가능합니다. 이는 국내 해양 환경 보호를 위한 중요한 조치 중 하나입니다.",
    "QUESTION": "지방해양수산청에서는 어떤 경우에 선박의 출항을 허가하나요?",
    "ANSWER": "지방해양수산청에서는 출항 전까지 제출된 자료를 검토하여 미배출 여부를 확인하고, 선박의 출항은 배출 여부 확인 후에 가능합니다."
    }},
    {{
    "CONTEXT": "국내 해양수산 당국은 방사능 오염 위험을 줄이기 위해 여러 가지 예방 조치를 시행하고 있습니다. 특히 일본 동부 6개현에서 출발하는 선박의 경우, 평형수 관리가 더욱 엄격하게 이루어집니다. 이러한 선박은 국내 입항 24시간 전까지 평형수 입항보고서를 제출해야 하며, 우리나라 해역으로 들어오기 전, 즉 관할수역 밖에서 평형수를 교환해야 합니다. 이를 통해 방사능 오염 물질이 국내 해역으로 유입되는 것을 방지하고 있습니다.",
    "QUESTION": "방사능 오염 선박평형수가 국내에 유입되지 않도록 어떤 조치가 시행되고 있습니까?", 
    "ANSWER": "일본 동부 6개현에서 평형수를 주입 후 국내 입항 예정인 선박은 국내 입항 24시간 전까지 평형수 입항보고서를 제출해야 하며, 일본 동부 6개현에서 평형수를 주입 후 국내 배출 예정인 선박은 우리나라 관할수역 밖에서 평형수 교환 후 입항해야 합니다."}}
    ```
    """
)

parser = JsonOutputParser(pydantic_object=QAPair)

# Combine prompt and parser into a chain
chain = (
    prompt_template
    | ChatOpenAI(
        model="gpt-4",
        temperature=0,
        streaming=True,
        callbacks=[StreamingStdOutCallbackHandler()],
    )
    | parser
)

qa_pair = []

print(len(texts))

for doc in texts:
    if doc.page_content:
        qa_pair.append(
            chain.invoke(
                {"context": doc.page_content, "domain": "포트미스 항만 포털 가이드북 내 자료", "num_questions": "1"}
            )
        )

print(qa_pair)


In [ ]:
import json
from datetime import datetime

num_qa_pairs = len(qa_pair)
current_date = datetime.now().strftime("%Y%m%d")
filename = f"{file.split('/')[-1]}_{num_qa_pairs}_{current_date}.json"

# Save as JSON file
with open(filename, "w", encoding="utf-8") as json_file:
    json.dump(qa_pair, json_file, ensure_ascii=False, indent=4)

print(f"QA pairs have been saved to {filename}")